In [8]:
import pandas as pd
import sqlite3
import os
from tabulate import tabulate

# Ensure output directory exists
os.makedirs('loaded', exist_ok=True)

# Load transformed CSVs
full_df = pd.read_csv('transformed/transformed_full.csv')
incremental_df = pd.read_csv('transformed/transformed_incremental.csv')

# Create SQLite database in loaded/ folder
db_path = 'loaded/etl_results.db'
conn = sqlite3.connect(db_path)

# Write DataFrames to SQLite tables
full_df.to_sql('full_data', conn, if_exists='replace', index=False)
incremental_df.to_sql('incremental_data', conn, if_exists='replace', index=False)

# Preview the stored results using SQL query and format as table
preview_df = pd.read_sql_query('SELECT * FROM full_data LIMIT 5', conn)
# Reorder columns: Purpose after product, total_price after unit_price if both exist
cols = list(preview_df.columns)
if 'Purpose' in cols and 'product' in cols:
    cols.remove('Purpose')
    prod_idx = cols.index('product')
    cols.insert(prod_idx + 1, 'Purpose')
if 'total_price' in cols and 'unit_price' in cols:
    cols.remove('total_price')
    unit_idx = cols.index('unit_price')
    cols.insert(unit_idx + 1, 'total_price')
preview_df = preview_df[cols]
print('Preview of full_data table:')
print(tabulate(preview_df, headers='keys', tablefmt='grid'))

# Close the connection
conn.close()

Preview of full_data table:
+----+------------+-----------------+-----------+-----------+------------+--------------+---------------+--------------+----------+
|    |   order_id | customer_name   | product   | Purpose   |   quantity |   unit_price |   total_price | order_date   | region   |
+====+============+=================+===========+===========+============+==============+===============+==============+==========+
|  0 |          1 | Diana           | Tablet    | Other     |          2 |          500 |          1000 | 2024-01-20   | South    |
+----+------------+-----------------+-----------+-----------+------------+--------------+---------------+--------------+----------+
|  1 |          2 | Eve             | Laptop    | Leisure   |          2 |          500 |          1000 | 2024-04-29   | North    |
+----+------------+-----------------+-----------+-----------+------------+--------------+---------------+--------------+----------+
|  2 |          3 | Charlie         | Laptop    